# CTGAN as a sampling technique for LIME and SHAP

* Fooling LIME and SHAP: https://github.com/dylan-slack/Fooling-LIME-SHAP/blob/master/COMPAS_Example.ipynb
* CTGAN: https://github.com/sdv-dev/CTGAN

### Progress

* If CTGAN is trained properly, we can identify about 43% of biased predictions (out of 1500 test samples from COMPAS)

In [32]:
!pip install lime shap

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 535.6/535.6 kB 28.1 MB/s eta 0:00:00


In [12]:
!pip install ctgan

Defaulting to user installation because normal site-packages is not writeable


In [37]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from copy import deepcopy
import sys

sys.path.append('../experiments/Fooling-LIME-SHAP-master/')

from adversarial_models import Adversarial_Lime_Model
from utils import one_hot_encode
import pandas as pd
import lime
import lime.lime_tabular

os.getcwd()

'/home/ans037/private/Faster-LIME/notebooks'

In [2]:
!python --version

Python 3.9.5


In [ ]:
# !sudo apt-get install python3.7

# !sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.7 1

# !sudo update-alternatives --config python3

# !sudo apt install python3-pip

# !sudo apt-get install python3.7-distutils

In [ ]:
# !pip install ctgan


In [ ]:
# !pip install --upgrade pip setuptools


In [ ]:
# !pip install numpy torch torchvision lime

In [ ]:
# !pip install -r requirements-experiment.txt

In [ ]:
# import os

# import torch
# import sys
# torch.manual_seed(42)

# import numpy as np
# from scipy.spatial.distance import cdist
# from sklearn.linear_model import Ridge
# from sklearn.preprocessing import StandardScaler, OneHotEncoder
# from sklearn.model_selection import train_test_split
# from copy import deepcopy
# sys.path.append('/experiments/Fooling-LIME-SHAP/')

# from adversarial_models import Adversarial_Lime_Model
# from utils import one_hot_encode
# import pandas as pd
# import lime
# import lime.lime_tabular

# os.environ['CUDA_VISIBLE_DEVICES'] = '1'

# sys.path.append('/experiments/CTGAN')

# from ctgan import load_demo
# from tqdm import tqdm

# data = load_demo()
# data

In [ ]:
discrete_columns = [
    'workclass',
    'education',
    'marital-status',
    'occupation',
    'relationship',
    'race',
    'sex',
    'native-country',
    'income'
]

In [ ]:
# from ctgan import CTGANSynthesizer

# ctgan = CTGANSynthesizer(batch_size=500, 
#                          gen_dim=[256,256],
#                          dis_dim=[256,256],
#                          l2scale=0, gen_lr=2e-8, dis_lr=2e-8)
# ctgan.fit(data, discrete_columns, epochs=30)

In [ ]:
# ctgan.sample(1000)

In [3]:
import os
os.getcwd()

'/home/ans037/private/Faster-LIME/notebooks'

# Always getting nans on adult dataset, try with the COMPAS dataset

In [4]:

from dataset_utils import get_and_preprocess_compas_data

data = get_and_preprocess_compas_data()

In [5]:
X, y, cols = data['data'], data['target'], data['cols']
cols

['age',
 'two_year_recid',
 'priors_count',
 'length_of_stay',
 'c_charge_degree_F',
 'c_charge_degree_M',
 'sex_Female',
 'sex_Male',
 'race']

In [6]:
print(X.shape)
print(y.shape)

(6172, 9)
(6172,)


In [7]:
X

,age,two_year_recid,priors_count,length_of_stay,c_charge_degree_F,c_charge_degree_M,sex_Female,sex_Male,race
id,,,,,,,,,
1,69,0,0,0,True,False,False,True,False
3,34,1,0,10,True,False,False,True,True
4,24,1,4,1,True,False,False,True,True
7,44,0,0,1,False,True,False,True,False
8,41,1,14,6,True,False,False,True,False
...,...,...,...,...,...,...,...,...,...
10996,23,0,0,1,True,False,False,True,True
10997,23,0,0,1,True,False,False,True,True
10999,57,0,0,1,True,False,False,True,False


# Create an explainer around the ctgan sampler

In [ ]:
# sys.path.append('../faster_lime/')

# def ridge_solve(tup):
#     data_synthetic_onehot, model_pred, weights = tup
#     solver = Ridge(alpha=1, fit_intercept=True)
#     solver.fit(data_synthetic_onehot,
#                model_pred,
#                sample_weight=weights.ravel())
    
#     # Get explanations
#     importance = solver.coef_[
#         data_synthetic_onehot[0].toarray().ravel() == 1].ravel()
#     return importance

# class NumpyTabularExplainer:

#     def __init__(self, training_data, ctgan_sampler, feature_names=None,
#                  categorical_feature_idxes=None,
#                  qs=[25, 50, 75], **kwargs):
#         """
#         Args:
#             training_data:
#             feature_names:
#             categorical_feature_idxes:
#             qs:
#             **kwargs:

#         Assumptions:
#             * Data only contains categorical and/or numerical data
#             * Categorical data is already converted to ordinal labels (e.g. via scikit-learn's
#                 OrdinalEncoder)

#         """
#         self.training_data = training_data
#         self.num_features = self.training_data.shape[1]
#         self.ctgan_sampler = ctgan_sampler

#         # Parse columns
#         if feature_names is not None:
#             # TODO input validation
#             self.feature_names = list(feature_names)
#         else:
#             self.feature_names = list(range(self.num_features))
#         self.categorical_feature_idxes = categorical_feature_idxes
#         if self.categorical_feature_idxes:
#             self.categorical_features = [self.feature_names[i] for i in
#                                          self.categorical_feature_idxes]
#             self.numerical_features = list(set(self.feature_names) - set(self.categorical_features))
#             self.numerical_feature_idxes = [idx for idx in range(self.num_features) if
#                                             idx not in self.categorical_feature_idxes]
#         else:
#             self.categorical_features = []
#             self.numerical_features = self.feature_names
#             self.numerical_feature_idxes = list(range(self.num_features))

#         # Some book-keeping: keep track of the original indices of each feature
#         self.dict_feature_to_idx = {feature: idx for (idx, feature) in
#                                     enumerate(self.feature_names)}
#         self.list_reorder = [self.dict_feature_to_idx[feature] for feature in
#                              self.numerical_features + self.categorical_features]

#         # Get training data statistics
#         # Numerical feature statistics
#         if self.numerical_features:
#             training_data_num = self.training_data[:, self.numerical_feature_idxes]
#             self.sc = StandardScaler(with_mean=False)
#             self.sc.fit(training_data_num)
#             self.qs = qs
#             self.all_bins_num = np.percentile(training_data_num, self.qs, axis=0).T

#         # Categorical feature statistics
#         if self.categorical_features:
#             training_data_cat = self.training_data[:, self.categorical_feature_idxes]
#             self.dict_categorical_hist = {
#                 feature: np.bincount(training_data_cat[:, idx]) / self.training_data.shape[0] for
#                 (idx, feature) in enumerate(self.categorical_features)
#             }

#     def kernel_fn(self, distances, kernel_width):
#         return np.sqrt(np.exp(-(distances ** 2) / kernel_width ** 2))

#     def discretize(self, X, qs=[25, 50, 75], all_bins=None):
#         if all_bins is None:
#             all_bins = np.percentile(X, qs, axis=0).T
#         return (np.array([np.digitize(a, bins)
#                           for (a, bins) in zip(X.T, all_bins)]).T, all_bins)
    
#     def explain_instance(self, data_row, predict_fn, num_estimators=10, label=0, num_samples=5000, num_features=10,
#                          kernel_width=None, **kwargs):
#         # Scale the data
#         data_row = data_row.reshape((1, -1))
        
#         # Sample data using the CTGAN
#         data_samples = None
#         while data_samples is None:
#             try:
#                 data_samples = self.ctgan_sampler.sample(int(num_estimators) * int(num_samples)).values
#             except:
#                 data_samples = None
                
#         for batch_idx in range(num_estimators):
#             data_samples[batch_idx * num_samples] = data_row.ravel()

#         # Split data into numerical and categorical data and process
#         list_orig = []
#         list_disc = []
#         if self.numerical_features:
#             data_num_synthetic = data_samples[:, self.numerical_feature_idxes]
#             # Discretize
#             data_synthetic_num_disc, _ = self.discretize(data_num_synthetic, self.qs,
#                                                          self.all_bins_num)
#             list_disc.append(data_synthetic_num_disc)
#             list_orig.append(data_num_synthetic)

#         if self.categorical_features:
#             # Sample from training distribution for each categorical feature
#             data_cat_synthetic = data_samples[:,self.categorical_feature_idxes]
#             list_disc.append(data_cat_synthetic)
#             list_orig.append(data_cat_synthetic)

#         # Concatenate the data and reorder the columns
#         data_synthetic_original = np.concatenate(list_orig, axis=1)
#         data_synthetic_disc = np.concatenate(list_disc, axis=1)
#         data_synthetic_original = data_synthetic_original[:, self.list_reorder]
#         data_synthetic_disc = data_synthetic_disc[:, self.list_reorder]

#         # Get model predictions (i.e. groundtruth)
#         model_pred = predict_fn(data_synthetic_original)

#         # Get distances between original sample and neighbors
# #         if self.numerical_features:
# #             distances = cdist(data_num_synthetic[:1], data_num_synthetic).reshape(-1, 1)
# #         else:
#         distances = cdist(data_synthetic_disc[:1], data_synthetic_disc).reshape(-1, 1)

#         # Weight distances according to some kernel (e.g. Gaussian)
#         if kernel_width is None:
#             kernel_width = np.sqrt(data_row.shape[1]) * 0.75
#         weights = self.kernel_fn(distances, kernel_width=kernel_width).ravel()

#         # Turn discretized data into onehot
#         data_synthetic_onehot = OneHotEncoder().fit_transform(data_synthetic_disc)

#         batch_size = num_samples
#         importances = []

#         iterator = ((data_synthetic_onehot[batch_idx * batch_size:(batch_idx + 1) * batch_size],
#                      model_pred[batch_idx * batch_size:(batch_idx + 1) * batch_size, label],
#                      weights[batch_idx * batch_size:(batch_idx + 1) * batch_size]) for batch_idx
#                     in range(num_estimators))

#         for tup in iterator:
#             # Solve
#             importance = ridge_solve(tup)
#             importances.append(importance)
        
#         importances = np.mean(np.stack(importances), axis=0)
# #         # Solve
# #         solver = Ridge(alpha=1, fit_intercept=True)
# #         solver.fit(data_synthetic_onehot, model_pred[:, label], sample_weight=weights)

# #         # Get explanations
# #         importances = solver.coef_[data_synthetic_onehot[0].toarray().ravel() == 1]
#         explanations = sorted(list(zip(self.feature_names, importances)),
#                               key=lambda x: x[1], reverse=True)[:num_features]
#         return explanations

In [9]:
X

,age,two_year_recid,priors_count,length_of_stay,c_charge_degree_F,c_charge_degree_M,sex_Female,sex_Male,race
id,,,,,,,,,
1,69,0,0,0,True,False,False,True,False
3,34,1,0,10,True,False,False,True,True
4,24,1,4,1,True,False,False,True,True
7,44,0,0,1,False,True,False,True,False
8,41,1,14,6,True,False,False,True,False
...,...,...,...,...,...,...,...,...,...
10996,23,0,0,1,True,False,False,True,True
10997,23,0,0,1,True,False,False,True,True
10999,57,0,0,1,True,False,False,True,False


## Follow along with what the attack did

In [13]:
# Add a random column -- this is what we'll have LIME/SHAP explain.
X['unrelated_column'] = np.random.choice([0,1],size=X.shape[0])
features = [c for c in X]

categorical_feature_name = ['two_year_recid', 'c_charge_degree_F', 'c_charge_degree_M',\
                            'sex_Female', 'sex_Male', 'race', 'unrelated_column']

categorical_feature_indcs = [features.index(c) for c in categorical_feature_name]

race_indc = features.index('race')
unrelated_indcs = features.index('unrelated_column')

In [15]:
### Train the ctgan model

from ctgan import CTGAN

ctgan = CTGAN()
ctgan.fit(X, categorical_feature_name, epochs=100)

In [20]:
# We made this shitlol

ctgan.sample(1000)

,age,two_year_recid,priors_count,length_of_stay,c_charge_degree_F,c_charge_degree_M,sex_Female,sex_Male,race,unrelated_column
0,23,0,1,17,True,False,False,True,True,1
1,23,0,0,-1,False,True,True,False,False,0
2,20,0,1,0,True,True,False,True,True,1
3,30,1,0,-1,True,False,False,True,True,0
4,34,0,0,1,False,True,True,False,False,1
...,...,...,...,...,...,...,...,...,...,...
995,17,0,1,30,True,False,False,True,True,0
996,26,0,1,12,False,True,False,True,True,0
997,33,0,0,10,False,True,False,True,False,0
998,65,1,5,34,True,False,False,True,True,0


In [18]:
class racist_model_f:
    # Decision rule: classify negatively if race is black
    def predict(self,X):
        return np.array([0 if x[race_indc] > 0 else 1 for x in X])

    def predict_proba(self, X): 
        return one_hot_encode(self.predict(X))

    def score(self, X,y):
        return np.sum(self.predict(X)==y) / len(X)
    
class innocuous_model_psi:
    # Decision rule: classify according to randomly drawn column 'unrelated column'
    def predict(self,X):
        return np.array([0 if x[unrelated_indcs] > 0 else 1 for x in X])

    def predict_proba(self, X): 
        return one_hot_encode(self.predict(X))

    def score(self, X,y):
        return np.sum(self.predict(X)==y) / len(X)

In [34]:
if type(X) is pd.DataFrame:
    X = X.values

# Split the data and normalize
xtrain,xtest,ytrain,ytest = train_test_split(X,y)
xtest_not_normalized = deepcopy(xtest)
# ss = StandardScaler().fit(xtrain)
# xtrain = ss.transform(xtrain)
# xtest = ss.transform(xtest)

# Train the adversarial model for LIME with f and psi 
adv_lime = Adversarial_Lime_Model(racist_model_f(), innocuous_model_psi()).\
            train(xtrain, ytrain, feature_names=features, categorical_features=categorical_feature_indcs)

In [38]:
import lime
import lime.lime_tabular

# Let's just look at a the first example in the test set
ex_indc = np.random.choice(xtest.shape[0])


# Now, lets look at the explanations on the adversarial model 
adv_explainer = lime.lime_tabular.LimeTabularExplainer(xtrain,feature_names=adv_lime.get_column_names(), 
                                                       discretize_continuous=False,
                                                       categorical_features=categorical_feature_indcs)

adv_exp = adv_explainer.explain_instance(xtest[ex_indc], adv_lime.predict_proba, labels=(0,1)).as_list(1)

print ("Explanation on adversarial model:\n",adv_exp[:3],"\n")

Explanation on adversarial model:
 [('unrelated_column=0', 0.996305140022845), ('two_year_recid=1', 0.0007813020007031841), ('race=1', -0.0007425201030233886)] 



In [ ]:
numpy_explainer = NumpyTabularExplainer(
    training_data=xtrain,
    ctgan_sampler=ctgan,
    feature_names=features,
    categorical_feature_idxes=categorical_feature_indcs
)

In [ ]:
# %timeit numpy_explainer.explain_instance(xtest[ex_indc], adv_lime.predict_proba, label=1, num_samples=1000, num_features=3)

In [ ]:
numpy_explainer.explain_instance(xtest[ex_indc], adv_lime.predict_proba, label=1, num_samples=5000, num_features=3)

In [ ]:
list_exp = []
for idx in tqdm(range(xtest.shape[0]), total=xtest.shape[0]):
    exp = numpy_explainer.explain_instance(xtest[idx], adv_lime.predict_proba, label=1, num_samples=5000, num_features=3)
    top = exp[0][0]
    list_exp.append(top)

In [ ]:
print(len([a for a in list_exp if a == 'race']) / len(list_exp))
